In [ ]:
from mutagen.mp4 import MP4
from pathlib import Path
import re

In [ ]:
input_path = Path.cwd() / 'input'

In [ ]:
input_folder = 'Watch - TV Shows (Anime)'
input_path = Path('d:/') / input_folder # f in asus, d in lenovo

In [ ]:
folder_list = [x for x in input_path.iterdir() if x.is_dir()]

In [ ]:
def update_title(file_path):
    file = MP4(file_path)
    title = ''.join(file['©nam'])
    
    if not title:
        file['©nam'] = file_path.stem
        file['©cmt'] = file_path.stem
        file.save()

In [ ]:
def get_season(folder_path):
    pattern = r'\[S\d+\]'
    result = re.search(pattern, folder_path.name)
    if result:
        text = result.group()
        search = re.search(r'\d+', text)
        season = search.group().zfill(2)
    else:
        season = '1'.zfill(2)
    return season

In [ ]:
def get_episode(file_path):
    patterns = [
        r'\[E\d+\]',
        r'\[\d+(v\d)?\]',
        r'\[\d+\s?END\]',
        r'\[\d+_\d+\]',
        r'-\s?\d+',
        r'\.\d+',
        r'第\d+话',
        r'第\d+話'
    ]
    
    for pattern in patterns:
        result = re.search(pattern, file_path.stem)
        if result:
            text = result.group()
            search = re.search(r'\d+', text)
            episode = search.group()
            return episode

In [ ]:
def get_episodes(file_list):
    episodes = []
    for file_path in file_list:
        keywords = ['.5', 'OVA']
        for keyword in keywords:
            if keyword in file_path.stem:
                episode = None
                break
        else:
            episode = get_episode(file_path)
        episodes.append(episode)
    
    num_digit = max([len(x) for x in episodes if x])
    episodes = [x.zfill(num_digit) if x else x for x in episodes]
    return episodes

In [ ]:
for folder_path in folder_list:
    file_list = list(folder_path.glob('*.mp4'))
    for file_path in file_list:
        update_title(file_path)
    
    pattern = r'\[(.*?)\]'
    result = re.findall(pattern, folder_path.name)
    
    english = result[0]
    chinese = result[1]
    subtitle = result[-3]
    resolution = result[-2]

    print(folder_path.name)
    season = get_season(folder_path)
    episodes = get_episodes(file_list)
    for episode in episodes:
        if episode:
            name = f'{english}.{chinese}.S{season}E{episode}.{subtitle}.{resolution}.mp4'
            print(name)
    print()